In [1]:
import folium
import pandas as pd
import numpy as np
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
fp = 'data/shopping_centers.txt'
data = pd.read_csv(fp, sep=';') 

print(data)

   ID         Name                                 addr
0   1         Itis          Itäkatu 1-7, 00930 Helsinki
1   2        Forum   Mannerheimintie 20, 00100 Helsinki
2   3     IsoOmena         Piispansilta 11, 02230 Espoo
3   4    Myyrmanni            Iskoskuja 3, 01600 Vantaa
4   5        Jumbo    Vantaanportinkatu 3, 01510 Vantaa
5   6         Dixi             Ratatie 11, 01300 Vantaa
6   7   Ruoholahti      Itämerenkatu 21, 00180 Helsinki


In [3]:
from geopandas.tools import geocode

# Geocode the addresses using Nominatim
geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_CM', timeout=10)

KeyError: 'addr'

In [ ]:
geo

In [ ]:
# Convert points to GeoJSON
geo_gjson = folium.features.GeoJson(geo, name="Shopping centers")

In [ ]:
# Create a Map instance
m = folium.Map(location=[60.20, 24.96], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

shopping_centers = ['Itis', 'Forum', 'IsoOmena', 'Sello', 'Jumbo', 'REDI', 'Tripla']
latitudes = [60.21170, 60.16893, 60.16294, 60.21752, 60.29245, 60.18702, 60.19801]
longitudes = [25.08294, 24.93858, 24.73779, 24.81042, 24.96282, 24.97904, 24.93440]

coords = list(zip(latitudes, longitudes))

from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster(coords, name='Shopping centers', popups=shopping_centers, color='green').add_to(m)

#Show map
m

In [ ]:
# Read text file into a variable called grid
grid = gpd.read_file('data/MetropAccess_YKR_grid_EurefFIN.shp')

# Check the data
grid

In [ ]:
# Import module
import glob

# Fetch the filepaths to all textfiles found in data/ 
fp = r'dataM/'

# Create a list for all textfiles found in data/
filepaths = glob.glob(fp + "/*.txt")

In [ ]:
# Create a for loop to iterate over the files in filepaths
for fp in filepaths:
    
    # Read the data
    data = pd.read_csv(fp, sep=';')
    
    # Select from_id and pt_r_t columns
    data = data[['from_id', 'pt_r_t']]
    
    # Rename from_id column to YKR_ID
    data = data.rename(columns={'from_id': 'YKR_ID'})
    
    # Create a variable to store shopping center names, obtained by splitting and and slicing the file names
    shopping_center_name = fp.split("_")[-1][:-4] 
    
    # Rename pt_r_t to include the shopping center names
    data = data.rename(columns={'pt_r_t': 'pt_r_t_' + shopping_center_name})
    
    # Merge the grid GeoDataFrame and the travel times using YKR_ID
    grid = grid.merge(data, on="YKR_ID")

In [ ]:
# Remove no data values
data[data==-1]=np.nan
data.dropna(inplace=True)

In [ ]:
# Create a new column for minimum travel time values 
grid['min_t'] = None

# Make a for loop to iterate over the rows
for i, row in grid.iterrows():
    
    # Create a variable to store all the columns needed
    value_columns = ['pt_r_t_Jumbo', 'pt_r_t_Dixi',
       'pt_r_t_Myyrmanni', 'pt_r_t_Itis', 'pt_r_t_Forum', 'pt_r_t_IsoOmena',
       'pt_r_t_Ruoholahti']
    
    # Find out the shortest travel time to any shopping center
    grid.loc[i, "min_t"] = row[value_columns].astype(float).min()
  
# Create a new column for the closest shopping center id values
grid['dominant_service'] = None

# Make a for loop to iterate over the rows
for i, row in grid.iterrows():
    
    # Create a variable to store all the columns needed
    value_columns = ['pt_r_t_Jumbo', 'pt_r_t_Dixi',
       'pt_r_t_Myyrmanni', 'pt_r_t_Itis', 'pt_r_t_Forum', 'pt_r_t_IsoOmena',
       'pt_r_t_Ruoholahti']
    # Find out the column name for the shortest travel time
    grid.loc[i, "dominant_service"] = row[value_columns].astype(float).idxmin()
    
grid

In [ ]:
for i, row in grid.iterrows():
    if grid.loc[i, "dominant_service"] == "pt_r_t_Myyrmanni":
        grid.loc[i, "sc"] = 1
    if grid.loc[i, "dominant_service"] == "pt_r_t_Dixi":
        grid.loc[i, "sc"] = 2
    if grid.loc[i, "dominant_service"] == "pt_r_t_Forum":
        grid.loc[i, "sc"] = 3
    if grid.loc[i, "dominant_service"] == "pt_r_t_IsoOmena":
        grid.loc[i, "sc"] = 4
    if grid.loc[i, "dominant_service"] == "pt_r_t_Itis":
        grid.loc[i, "sc"] = 5
    if grid.loc[i, "dominant_service"] == "pt_r_t_Jumbo":
        grid.loc[i, "sc"] = 6
    if grid.loc[i, "dominant_service"] == "pt_r_t_Ruoholahti":
        grid.loc[i, "sc"] = 7

In [ ]:
grid["geoid"] = grid.index.astype(str)

In [ ]:
# Plot a choropleth map
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=grid,
    name='Dominances',
    data=grid,
    columns = ['geoid', 'sc'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Minimum distance').add_to(m)

#Show map
m